# HW8.1 Text classification (sentiment analysis) with deep learning models

In this homework, we will practice building various deep learning model architectures for text classification. We will be using the IMBD movie review data for the sentiment classification task. First we will load the data. Second, you will be able to use various model types we've learned so far to perform text classification. 

Remember to use GPUs for this one, otherwise it will be slow to train. 

Tips:
- print out the data types and shapes after each step to verify you are doing what you expected to do.
- when you build the model, pass in the validation data in `model.fit()` as an argument `validation_data=(X_val, Y_val)`. The test data should be reserved until after you trained the model and then you can test it with X_test and report the test accuracy. **Always report accuracy on Test data for each experiment you do and make the comparison based on Test data.**

## 1. Loading data

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling1D
# fix random seed for reproducibility
tf.random.set_seed(7)

### Preprocessing data

There are a few hyperparameters you have to set and then preprocess the text data accordingly. These hyperparameters are:

1. `vocab_size` : if set to 5000, the model will only keep the number of top 5000 most frequent words in the vocablulary while processing the text. Any less frequent words are thrwon out. 
2. `max_review_length` : if set to 500, the model will keep the maxmium length of a review text at 500. Any reviews longer than 500 words will be truncated at 500 words, and any reviews shorter than 500 words will be padded with 0s or special padding token to match the 500 token length. This is to makes sure all input sentences during training in a batch have the same size, a requirement of the batched input. 
3. `embedding_vector_length` : this is the embedding vector you want to project each word too. For instance, we can set it to 256 or 512. 



In [ ]:

vocab_size = 5000
max_review_length = 500

# load the dataset but only keep the top n words with the argument num_words
# please read documentation here: https://keras.io/api/datasets/imdb/
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)



### task 0: review length before and after padding

print out the length of first 20 reviews below. Then execute the next cell to pad them and then print out again the lengths. what do you see?

In [ ]:
# YOUR CODE HERE

In [ ]:
# truncate and pad input sequences
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# YOUR CODE HERE 

### task 1: split training data and label into train and validation sets, X_train and X_val. 

Use 20% of your training data for validation, and the rest 80% for training. Then, also pad your new training data and validation data the same way as you did for train and test. You can leave the Test data alone and reserve that portion for testing only after training the model. 

In [ ]:
# YOUR CODE HERE

### Understanding the text data format

The text data for deep learning is represented in a way that each word is mapped into a integer index first, 1, 2, 3, ..., N, assuming the text has N words in the vocabulary. Then we construct a giant embedding matrix where each word has an entry. At training time, you just use the index to retrieve the word embedding correspond to that entry (such as the k-th embedding) from this matrix. Let's inspect the input data representation before it is projected into the embedding space. 

In [ ]:
print(X_train.shape, y_train.shape, 'train sequences')
print(X_val.shape, y_val.shape,'val sequences')
print(X_test.shape, y_test.shape, 'test sequences')

In [ ]:
# inspect the 24rd review: 
print(X_train[23])

# inspect the 2nd review: 
print(X_train[1])

### Q1: Do you see the difference between these two examples in terms of padding? explain what is happening. 

In [ ]:
# YOUR ANSWER HERE

In [ ]:
# now let's look at these reviews in actual words 

INDEX_FROM = 3
word_index = imdb.get_word_index()
word_index = {key:(value+INDEX_FROM) for key,value in word_index.items()}
word_index["<PAD>"] = 0    # the padding token
word_index["<START>"] = 1  # the starting token
word_index["<UNK>"] = 2    # the unknown token
reverse_word_index = {value:key for key, value in word_index.items()}

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(X_train[23])

## 2. Build models 

In this homework you will demonstrate your ability to build various kinds of models for sequence (text) classification. Specifically: 

- Using single architectures: 
  - CNN 1d layer (https://keras.io/api/layers/convolution_layers/convolution1d/)
  - LSTM (https://keras.io/api/layers/recurrent_layers/lstm/)
  - Bidirectional LSTM (https://keras.io/api/layers/recurrent_layers/bidirectional/) (for this one, you want to do something like `Bidirectional(LSTM(num_units))`)

- Stacking these layers together: Conv-LSTM, Conv-BiLSTM: it just means once you have your conv1d layers, add another (or several) LSTM or Bidirectional LSTM on top of it. 

#### 2.1 Conv1d 

Tips: to get started, first build a Sequential model. Then add a `Embedding` layer (https://keras.io/api/layers/core_layers/embedding/) with the `input_dim` equal to your vocab_size, and the `output_dim` equal to your `embedding_vector_length`. You should also add an argument `input_length` being equal to your `max_review_length`. Then add a conv1d layer with multiple filters (maybe 64), then a `MaxPooling1D` layer with a pooling factor of 2. You can feel free to repeat this structure another 1 to 3 times if you want. Then before you go into the Dense layer, you need to `Flatten` the output from Conv layers. Once you flattened the output, you can add either another (nonlinear) Dense layer with some units (such as 128) or not, before you add the final Dense layer with a sigmoid activation. 

Overall the flow is: 

Embedding -> (Conv1D->MaxPooling1D) * K times -> Flatten -> (Dense with relu activation) * M times -> output Dense layer with sigmoid activation. 

Note that K>=1 but M>=0. 

In [3]:
# task 2:CNN
# YOUR CODE HERE

#### 2.2 LSTM and Bi-LSTM

Once you built the network with CNN, this will be easy. Simply replace the Conv1d layer with the LSTM layer and the Bidirectional LSTM layer (read the documentations linked above).Try using 64 units for the LSTM layer and try 128 as well. 

In [ ]:
# task 3: LSTM and Bi-LSTM
# YOUR CODE HERE

#### 2.3 Conv-LSTM and Conv-BiLSTM

In this last task, you will stack together the Conv1d layers and the LSTM layers. Try adding the LSTM layer after the Conv1d layers and see if it works. You can try using two Conv1d layers and then add a LSTM layers. Then replace the LSTM layer with Bi-LSTM layer. Try different combinations and see what's the best accuracy you can get on Test data. 

In [ ]:
# task 3: Conv-LSTM
# YOUR CODE HERE

# Wrap up

Report the accuracies you got from different architectures and write down any insights you have learned. 